In [1]:
import pandas as pd
import numpy as np
import pymongo
import requests
import json
import pickle
from collections import defaultdict
from collections import Counter

In [2]:
client = pymongo.MongoClient("mongodb://localhost:37017" )
db=client["lol"]
coll=db["match_data"]

In [42]:
def lane_dicts(mongo_object):
    """

    """
    allied={}
    enemy={}
    winning_team=0
    bot_flag=[0,0]
    for i,player in enumerate(mongo_object["data"]["participants"]):

        if i<5:
            if bot_flag[0]==0 and player["timeline"]["lane"] == "BOTTOM":
                allied[player["timeline"]["lane"]+"2"]= player["championId"]
                bot_flag[0]=1
            else:
                allied[player["timeline"]["lane"]]= player["championId"]
        else:
            if bot_flag[1]==0 and player["timeline"]["lane"] == "BOTTOM":
                enemy[player["timeline"]["lane"]+"2"]= player["championId"]
                bot_flag[1]=1
            else:
                enemy[player["timeline"]["lane"]]= player["championId"]
    
            
    if mongo_object["data"]["teams"][0]["win"]=="Fail":
        winning_team=100
    else:
        winning_team=200
    return allied, enemy, winning_team

In [43]:
lane_dicts(x)

({u'BOTTOM': 18, u'BOTTOM2': 267, u'JUNGLE': 64, u'MIDDLE': 245, u'TOP': 98},
 {u'BOTTOM': 21, u'BOTTOM2': 25, u'JUNGLE': 141, u'MIDDLE': 4, u'TOP': 122},
 200)

In [60]:
def lane_matchups(mongo_object):
    allied,enemy,winning_team=lane_dicts(mongo_object)
    error_list=[]
    
    try:
        lane_matchups=Counter()
        lane_matchups_wins=Counter()
        for lane in allied:
            lane_matchups[(allied[lane],enemy[lane])]+=1
            if winning_team==100:
                lane_matchups_wins[(allied[lane],enemy[lane])]+=1
        lane_matchups[(allied["BOTTOM"],enemy["BOTTOM2"])]+=1
        lane_matchups[(allied["BOTTOM2"],enemy["BOTTOM"])]+=1
        if winning_team==100:
            lane_matchups_wins[(allied["BOTTOM"],enemy["BOTTOM2"])]+=1
            lane_matchups_wins[(allied["BOTTOM2"],enemy["BOTTOM"])]+=1

        for lane in enemy:
            lane_matchups[(enemy[lane],allied[lane])]+=1
            if winning_team==200:
                lane_matchups_wins[(enemy[lane],allied[lane])]+=1
        lane_matchups[(enemy["BOTTOM"],allied["BOTTOM2"])]+=1
        lane_matchups[(enemy["BOTTOM2"],allied["BOTTOM"])]+=1
        if winning_team==200:
            lane_matchups_wins[(enemy["BOTTOM"],allied["BOTTOM2"])]+=1
            lane_matchups_wins[(enemy["BOTTOM2"],allied["BOTTOM"])]+=1

        return lane_matchups, lane_matchups_wins
    except:
        
        error_list.append(mongo_object["_id"])
        return Counter({"666":1}),Counter({"666":1})#,error_list
        

In [64]:
def total_and_wins_for_lane_matchup(iterable_games_data):
    """
    Calls and aggregates results of lane_matchups() over 
    iterable collection of games data.
    Returns two counter dicts
    """
    lane_total_vs=Counter()
    lane_wins_vs=Counter()
    for game in iterable_games_data:
        lane_matchup, lane_matchup_wins=lane_matchups(game)
        lane_total_vs+=lane_matchup
        lane_wins_vs+=lane_matchup_wins
    return lane_total_vs,lane_wins_vs

In [65]:
for x in range(10):
    
    print list(coll.find({"_id":10}))[0]["data"]["participants"][x]["timeline"]["lane"]

MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE
MIDDLE


In [69]:
import timeit

In [77]:
def time_me():
    return total_and_wins_for_lane_matchup(coll.find().limit(1000))

In [81]:
timeit.timeit(stmt=time_me,number=1)

9.290280818939209

In [91]:
x=list(coll.find({"_id":1}))[0]

In [82]:

total, wins = total_and_wins_for_lane_matchup(coll.find().limit(5000))

In [93]:
sorted(total.values(),reverse=True)

[2265,
 232,
 232,
 212,
 212,
 181,
 181,
 176,
 176,
 168,
 168,
 163,
 163,
 160,
 160,
 153,
 153,
 152,
 152,
 148,
 148,
 142,
 142,
 140,
 140,
 138,
 138,
 133,
 133,
 130,
 130,
 125,
 125,
 123,
 123,
 121,
 121,
 121,
 121,
 120,
 120,
 120,
 120,
 119,
 119,
 119,
 119,
 118,
 118,
 116,
 116,
 113,
 113,
 112,
 112,
 111,
 111,
 111,
 111,
 111,
 111,
 109,
 109,
 107,
 107,
 106,
 106,
 105,
 105,
 105,
 105,
 103,
 103,
 103,
 103,
 102,
 102,
 102,
 102,
 102,
 102,
 101,
 101,
 99,
 99,
 98,
 98,
 98,
 98,
 98,
 98,
 98,
 98,
 97,
 97,
 97,
 97,
 96,
 96,
 96,
 96,
 96,
 96,
 95,
 95,
 95,
 95,
 94,
 94,
 94,
 94,
 94,
 94,
 93,
 93,
 92,
 92,
 92,
 92,
 91,
 91,
 91,
 91,
 91,
 91,
 89,
 89,
 88,
 88,
 88,
 88,
 88,
 88,
 87,
 87,
 87,
 87,
 86,
 86,
 86,
 86,
 86,
 86,
 85,
 85,
 83,
 83,
 82,
 82,
 82,
 82,
 82,
 82,
 81,
 81,
 80,
 80,
 79,
 79,
 79,
 79,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 77,
 77,
 76,
 76,
 76,
 76,
 76,
 76,
 76,
 76,
 75,
 75,
 75,
 

In [92]:
lane_matchups(x)

(Counter({(4, 245): 1,
          (18, 21): 1,
          (18, 25): 1,
          (21, 18): 1,
          (21, 267): 1,
          (25, 18): 1,
          (25, 267): 1,
          (64, 141): 1,
          (98, 122): 1,
          (122, 98): 1,
          (141, 64): 1,
          (245, 4): 1,
          (267, 21): 1,
          (267, 25): 1}),
 Counter({(4, 245): 1,
          (21, 18): 1,
          (21, 267): 1,
          (25, 18): 1,
          (25, 267): 1,
          (122, 98): 1,
          (141, 64): 1}))